In [1]:
#| label: vfaFig6jn
from repo2data.repo2data import Repo2Data
import os 
import pickle
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from IPython.display import display, HTML
from plotly import tools

from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

with suppress_stdout():
    data_req_path = os.path.join("..","..","..", "binder", "data_requirement.json")
    repo2data = Repo2Data(data_req_path)
    DATA_ROOT = os.path.join(repo2data.install()[0],"t1-book-neurolibre")
    filename = os.path.join(DATA_ROOT,"02",'figure_6.pkl')

with open(filename, 'rb') as f:
    B1Range, mean_T1_noB1Correction, mean_T1_withB1Correction, std_T1_noB1Correction, std_T1_withB1Correction = pickle.load(f)

config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

data1 = dict(
        visible = True,
        x = B1Range,
        y = mean_T1_noB1Correction,
        name = 'Nominal flip angles',
        text = 'Nominal flip angles',
        mode = 'lines+markers',
        hoverinfo = 'x+y+text',
        line = dict(
            color = ('rgb(22, 96, 167)'),
            ),
        error_y=dict(
            type='data',
            array=std_T1_noB1Correction,
            visible=True,
            color = ('rgb(142, 192, 240)')
        ))

data2 = dict(
        visible = True,
        x = B1Range,
        y = mean_T1_withB1Correction,
        name = 'B<sub>1</sub>-corrected flip angles',
        text = 'B<sub>1</sub>-corrected flip angles',
        mode = 'lines+markers',
        hoverinfo = 'x+y+text',
        line = dict(
            color = ('rgb(205, 12, 24)'),
            ),
        error_y=dict(
            type='data',
            array=std_T1_withB1Correction,
            visible=True,
            color = ('rgb(248, 135, 142)')
        ))

data = [data1, data2]

layout = go.Layout(
    width=580,
    height=450,
    margin=go.layout.Margin(
        l=80,
        r=80,
        b=60,
        t=60,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.14,
            showarrow=False,
            text='B<sub>1</sub> (n.u.)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.17,
            y=0.5,
            showarrow=False,
            text='T<sub>1</sub> (s)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        autorange=False,
        range=[B1Range[0], B1Range[-1]],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=False,
        range=[0, max(mean_T1_noB1Correction)],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.32,
        y=0.98,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ),
    plot_bgcolor='white'
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'vfa_fig_6.html', config = config)
